In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Danuja
[nltk_data]     Wijerathna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Danuja
[nltk_data]     Wijerathna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
file_path = 'survey_combined -final.csv'
df = pd.read_csv(file_path)

In [6]:
df = df.dropna(how='all')

In [8]:
# Separate numerical and categorical fields
numerical_fields = ['Current Weight (kg)']
categorical_fields = ['Gender', 'Diuretics Medication', 'Beta-Blockers Medication', 'Medical Records Consent']
text_columns = [
    'Typical Workday in Heatwave',
    'Determine Water Intake',
    'Medication Effect on Hydration',
    'Feel When Dehydrated',
    'Severe Dehydration Experience',
    'Hydration Habits',
    'Tools for Monitoring Hydration'
]

# Function to preprocess text data
def preprocess_text(text):
    # Lowercasing
    text = str(text).lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenization (implicitly done by CountVectorizer later)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

# Enhanced extract_topics function with preprocessing
def extract_topics(data, n_topics=3, n_features=1000):
    # Preprocess the text data
    preprocessed_data = data.apply(preprocess_text)

    # Vectorization with TF-IDF to give less weight to more common words
    vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english')
    dtm = vectorizer.fit_transform(preprocessed_data)

    # Apply LDA
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(dtm)

    # Get the topic distribution for each document
    topic_distribution = lda.transform(dtm)
    return topic_distribution

for column in text_columns:
    topics = extract_topics(df[column])
    for i in range(topics.shape[1]):
        X[f'{column}_Topic_{i}'] = topics[:, i]

X= X.drop(columns=text_columns)



In [10]:
# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-Means clustering to the scaled features
kmeans_combined = KMeans(n_clusters=3, random_state=42)
kmeans_combined.fit(X_scaled)  # Use X_scaled directly

# Assign the clusters as the predicted body water levels with combined features
df['Predicted Body Water Level Combined'] = kmeans_combined.labels_

# Check the distribution of predicted body water levels with combined features
print(df['Predicted Body Water Level Combined'].value_counts())


1    48
2    29
0    23
Name: Predicted Body Water Level Combined, dtype: int64


In [ ]:
prompts = pd.read_csv('temp.csv')